In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import json
import numpy as np
import torch

def read_question_answer_file(file_path):
    """Reads a JSONL file with question-answer data and returns a list of dictionaries."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))  # Parse each line as JSON
    return data

# Load your dataset

dataset_path = 'med_data/phrases_no_exclude_train.jsonl'  # Replace with the path to your JSON file
questions_data = read_question_answer_file(dataset_path)

print(questions_data[0]['question'])
print(questions_data[0]['answer_idx'])
print(questions_data[0]['options'])

A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?
D
{'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Doxycycline', 'D': 'Nitrofurantoin'}


In [2]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.3 MB/s eta 0:00:0000:0100:01


In [3]:
from Bio import Entrez

# Replace with your email address
Entrez.email = "2935352R@student.gla.ac.uk"
access_key_id = "18acd1db794f7de35c1c83811bc106c6a509"

def fetch_pmids(query):
  """Fetches PMIDs for a given search term.

  Args:
    search_term: The search term to query Entrez.

  Returns:
    A list of PMIDs.
  """
  word = query.split()
#   if len(word) == 1:
#     query = query+" top 10 records"
  handle = Entrez.esearch(db="pubmed",sort="relevance", term=query, retmax=20,api_key=access_key_id)  # Adjust retmax as needed
  record = Entrez.read(handle)
  handle.close()

  pmids = record["IdList"]
  return pmids

# Example usage:
search_term = "Ceftriaxone"
pmids = fetch_pmids(search_term)
print(pmids)

['2210825', '1918224', '3910386', '3073046', '11985490', '27488586', '36208418', '37486410', '3906584', '37474675', '1918222', '1918223', '1918221', '29377708', '22761158', '19496200', '1486190', '15151561', '3433495', '35703558']


In [4]:
import requests

def fetch_bioc_data(pmid):
  """Fetches BioC data for a list of PMIDs.

  Args:
    pmids: A list of PMIDs.

  Returns:
    A list of JSON objects, each representing a BioC document.
  """

  #url = f"https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pubmed.cgi/BioC_json/{pmid}/unicode"
  url = f"https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/{pmid}/unicode"
  response = requests.get(url)
  if response.status_code == 200:
     try:
      bioc_data = response.json()
     except Exception:
      #print(f"Error decoding BioC data for PMID {pmid} (might be non-JSON)")
      bioc_data = None  # Or handle empty data differently
  else:
     print(f"Error fetching BioC data for PMID {pmid}: {response.status_code}")

  return bioc_data

# Example usage
pmid = "35703558"
bioc_data = fetch_bioc_data(pmid)
print(bioc_data)

[{'source': 'PMC', 'date': '20231103', 'key': 'pmc.key', 'infons': {}, 'documents': [{'id': '9430804', 'infons': {'license': 'CC BY'}, 'passages': [{'offset': 0, 'infons': {'article-id_doi': '10.1128/spectrum.00621-22', 'article-id_pmc': '9430804', 'article-id_pmid': '35703558', 'article-id_publisher-id': '00621-22 spectrum.00621-22', 'elocation-id': 'e00621-22', 'issue': '4', 'kwd': 'Enterococcus faecalis ampicillin ceftriaxone penicillin synergy', 'license': 'This is an open-access article distributed under the terms of the Creative Commons Attribution 4.0 International license.', 'name_0': 'surname:Cusumano;given-names:Jaclyn A.', 'name_1': 'surname:Khan;given-names:Ruhma', 'name_2': 'surname:Shah;given-names:Zeel', 'name_3': 'surname:Philogene;given-names:Cassie', 'name_4': 'surname:Harrichand;given-names:Amrita', 'name_5': 'surname:Huang;given-names:Vanthida', 'name_6': 'surname:Asempa;given-names:Tomefa E.', 'name_7': 'surname:Cusumano;given-names:Jaclyn', 'section_type': 'TITLE'

In [5]:
import re

def clean_text(text):
  """Removes special characters and extra whitespace from text.

  Args:
    text: The input text to be cleaned.

  Returns:
    The cleaned text.
  """
  special_chars = r"[\u03b2\u00b5\u03b4\u03c5\u03bb\u0394\u00f6]"
  # Remove special characters using regular expression
  text = re.sub(r"[^\w\s]", "", text)

  # Remove extra whitespace
  text = " ".join(text.split())

  return text

In [6]:
def save_string_to_file(data, filename):
  """Saves a string to a text file.

  Args:
    text: The string to be saved.
    filename: The name of the file to create.
  """

  with open(filename, "w",encoding='utf-8') as f:
    json.dump(data, f)

# # Example usage:
# my_string = "This is the text I want to save."
# save_string_to_file(my_string, "output.txt")

In [7]:
def sanitize_filename(filename):
  """Sanitizes a filename by replacing special characters with underscores.

  Args:
    filename: The original filename.

  Returns:
    The sanitized filename.
  """

  # Replace non-alphanumeric characters with underscores
  filename = re.sub(r'[^\w]', '_', filename)

  # Remove leading and trailing underscores
  filename = filename.strip('_')

  return filename

In [8]:
import json

def extract_text(data):
  """Extracts text from the given JSON data.

  Args:
    data: The JSON data.

  Returns:
    A list of text strings.
  """

  texts = []
  for document in data:
    for passage in document['documents'][0]['passages']:  # Access the first document's passages
        words = passage['text'].split()
        if(len(words)> 8):
          texts.append(clean_text(passage['text']))
  return texts

# Example usage:
# Assuming you have the JSON data stored in a variable named 'json_data'
extracted_texts = extract_text(bioc_data)
print(extracted_texts)

['Penicillin plus Ceftriaxone versus Ampicillin plus Ceftriaxone Synergistic Potential against Clinical Enterococcus faecalis Blood Isolates', 'Penicillin plus ceftriaxone is a promising alternative to ampicillin plus ceftriaxone for the treatment of Enterococcus faecalis infective endocarditis Limited data is available supporting the utilization of penicillin plus ceftriaxone A total of 20 E faecalis isolates one wildtype strain JH22 and 19 clinical blood strains were assessed for penicillin plus ceftriaxone and ampicillin plus ceftriaxone synergy using a 24h timekill experiment Susceptibility was determined by broth microdilution Differences in bactericidal bacteriostatic or inactivity as well as synergy between treatments were assessed by chisquare or Fisher exact test All E faecalis isolates were considered susceptible to ampicillin and penicillin Ampicillin plus ceftriaxone versus penicillin plus ceftriaxone similarly demonstrated synergy Bactericidal activity was more commonly ob

In [10]:
# Iterating a for loop to fetch the data.
import time

from tqdm import tqdm

questionLoader = questions_data[:200]
for question in  tqdm(questionLoader):
    options = question['options']
    for key, value in options.items():
        query = value
        bioc_data = []
        pmids = fetch_pmids(query)
        counter = 0
        for pmid in pmids:
            data = fetch_bioc_data(pmid)
            if data != None:
                counter +=1
                bioc_data.append(extract_text(data))
            time.sleep(0.3)
            if counter == 5 : break
        save_string_to_file(bioc_data, "Pubmed_Full_text/" + sanitize_filename(query) + ".json")

100%|██████████| 200/200 [3:14:03<00:00, 58.22s/it]  


In [36]:
questionLoader = questions_data[182:200]
questionLoader[0]

{'question': 'A 72-year-old man presents to his primary care provider to discuss the frequency with which he wakes up at night to urinate. He avoids drinking liquids at night, but the symptoms have progressively worsened. The medical history is significant for hypertension and hyperlipidemia. He takes lisinopril, atorvastatin, and a multivitamin every day. Today, the vital signs include: blood pressure 120/80 mm Hg, heart rate 90/min, respiratory rate 17/min, and temperature 37.0°C (98.6°F). On physical examination, he appears tired. The heart has a regular rate and rhythm and the lungs are clear to auscultation bilaterally. A bedside bladder ultrasound reveals a full bladder. A digital rectal exam reveals an enlarged and symmetric prostate free of nodules, that is consistent with benign prostatic enlargement. He also has a history of symptomatic hypotension with several episodes of syncope in the past. The patient declines a prostate biopsy that would provide a definitive diagnosis an

# Library installations

In [8]:
# !pip install langchain
# !pip install transformers
# !pip install accelerate
# !pip install bitsandbytes
# !pip install --upgrade pip
# !pip install --upgrade langchain
# !pip install langchain_community
# !pip list | grep langchain
# !pip list | grep langchain_community

# !pip install -U langchain-huggingface


   ---------------------------------------- 0.0/990.6 kB ? eta -:--:--
   --------------------- ------------------ 524.3/990.6 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 990.6/990.6 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: jsonpatch
    Found existing installation: jsonpatch 1.32
    Uninstalling jsonpatch-1.32:
      Successfully uninstalled jsonpatch-1.32


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.


   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   --------------------- ------------------ 524.3/991.5 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.6.1 which is incompatible.


   ---------------------------------------- 0.0/136.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/136.5 MB ? eta -:--:--
   ---------------------------------------- 1.0/136.5 MB 2.6 MB/s eta 0:00:52
   ---------------------------------------- 1.3/136.5 MB 2.2 MB/s eta 0:01:01
    --------------------------------------- 2.1/136.5 MB 2.6 MB/s eta 0:00:52
    --------------------------------------- 2.6/136.5 MB 2.8 MB/s eta 0:00:48
    --------------------------------------- 2.9/136.5 MB 2.3 MB/s eta 0:00:58
    --------------------------------------- 3.4/136.5 MB 2.3 MB/s eta 0:00:59
   - -------------------------------------- 3.9/136.5 MB 2.3 MB/s eta 0:00:57
   - -------------------------------------- 4.2/136.5 MB 2.4 MB/s eta 0:00:56
   - -------------------------------------- 4.5/136.5 MB 2.3 MB/s eta 0:00:59
   - -------------------------------------- 5.0/136.5 MB 2.2 MB/s eta 0:01:01
   - -------------------------------------- 5.2/136.5 MB 2.1 MB/s eta 0:01:03


   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.3 MB 985.5 kB/s eta 0:00:02
   --------- ------------------------------ 0.5/2.3 MB 985.5 kB/s eta 0:00:02
   ------------- -------------------------- 0.8/2.3 MB 729.2 kB/s eta 0:00:03
   ------------------ --------------------- 1.0/2.3 MB 967.3 kB/s eta 0:00:02
   --------------------------- ------------ 1.6/2.3 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 1.6 MB/s eta 0:00:00


'grep' is not recognized as an internal or external command,
operable program or batch file.
'grep' is not recognized as an internal or external command,
operable program or batch file.


   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   ------------------ --------------------- 1.0/2.2 MB 2.5 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 3.0 MB/s eta 0:00:01
   --------------------------------- ------ 1.8/2.2 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.5 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.5 MB 1.6 MB/s eta 0:00:06
   ------ --------------------------------- 1.6/9.5 MB 2.5 MB/s eta 0:00:04
   --------- ------------------------------ 2.4/9.5 MB 2.9 MB/s eta 0:00:03
   ------------ --------------------------- 2.9/9.5 MB 2.8 MB/s eta 0:00:03
   -------------- ------------------------- 3.4/9.5 MB 2.7 MB/s eta 0:00:03
   ---------------- --------------------

# Building the pipeline with the langchain

In [21]:
import os
import langchain

### prompts
from langchain import PromptTemplate, LLMChain

### models
# from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings


import torch
import transformers
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)


#model = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"


tokenizer = AutoTokenizer.from_pretrained(model)

        
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True,
)

model_llama = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config = bnb_config,
    device_map = 'auto'
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
question = questions_data[0]['question']
options = "\nA. Ampicillin\nB. Ceftriaxone\nC. Doxycycline\nD. Nitrofurantoin\n"

# Code design without a pipeline

In [22]:

templateX = """Question: {question}[INST]Select the correct option only. No explanation required[/INST]

Options:{options}

#Answer:"""  # Force a single-line response


prompt_template = PromptTemplate(template=templateX, input_variables=["question", "options"])
promptX = prompt_template.format(question=question, options=options) 




In [28]:
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token 
inputs = tokenizer(promptX, return_tensors='pt', truncation=True, padding="max_length", max_length=1024).to(model_llama.device)
outputs = model_llama.generate(**inputs, max_new_tokens=1)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?[INST]Select the correct option only. No explanation required[/INST]

Options:
A. Ampicillin
B. Ceftriaxone
C. Doxycycline
D. Nitrofurantoin


#Answer: D


In [26]:
position = response.find('#Answer:')
prediction = response[position+8 :position+10].strip()
prediction

''

# Designing the data loader for parsing the data faster

In [29]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from tqdm import tqdm  # For progress bar

class QuestionAnswerDataset(Dataset):
    def __init__(self, questions_data):
        self.questionData = questions_data
    

    def __len__(self):
        return len(self.questionData)

    def __getitem__(self, idx):
        question_data = self.questionData[idx]
        question = question_data['question']
        options = question_data['options']
        options_str = "\n".join([f"{key}. {value}" for key, value in options.items()])
        answer = question_data['answer_idx']
        return question,options_str,answer

# Load your dataset
dataset = QuestionAnswerDataset(questions_data)
dataloader = DataLoader(dataset, batch_size=8, shuffle=False)  # Adjust batch_size as needed



# Prompt design and generation

In [30]:
#prompt generation
template = """Question: {question}[INST]Select the correct option only. No explanation required[/INST]

Options:{options}

#Answer:"""  # Force a single-line response


prompt = PromptTemplate(template=template, input_variables=["question", "options"])

#llm_chain = LLMChain(llm=llm, prompt=prompt)

# Running the Batches

In [32]:
correct_predictions = 0
total_predictions = 0
responses = []
answers = []

for batch in tqdm(dataloader):
    questions, options_strs, answer_idxs = batch
    tokenizer.padding_side = 'left'
    tokenizer.pad_token = tokenizer.eos_token
    prompts = [prompt.format(question=question, options=options_str) for question, options_str in zip(questions, options_strs)]
    
    inputs = tokenizer(prompts, return_tensors='pt', padding=True, truncation=True, max_length=1500).to(model_llama.device)
    
    with torch.no_grad():
        outputs = model_llama.generate(**inputs,max_new_tokens=1)
    
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    #print(decoded_outputs)
    for decoded_output, answer in zip(decoded_outputs, answer_idxs):
        position = decoded_output.find('#Answer:')
        answer_pred = decoded_output[position+8 :position+10].strip()
        #print(answer_pred)
        if answer == answer_pred.strip():
            correct_predictions += 1
        
        responses.append(answer_pred)
        answers.append(answer)
        total_predictions += 1
   


100%|██████████| 1273/1273 [49:00<00:00,  2.31s/it]


In [33]:
print(f"Accuracy: {correct_predictions / len(responses):.2%}")
correct_predictions


Accuracy: 65.18%


6634

In [2]:
#!zip -r Pubmed_Full_text.zip "Pubmed_Full_text"
!unzip Pubmed_Abstract.zip

Archive:  Pubmed_Abstract.zip
   creating: Pubmed_Abstract/
 extracting: Pubmed_Abstract/.json   
  inflating: Pubmed_Abstract/0_45__saline_at_100_mL_h.json  
 extracting: Pubmed_Abstract/0_45__saline_with_30_mEq_L_KCl_at_100_mL_h.json  
  inflating: Pubmed_Abstract/0_56_to_1_88.json  
  inflating: Pubmed_Abstract/0_83_to_2_19.json  
 extracting: Pubmed_Abstract/100.json  
  inflating: Pubmed_Abstract/100__oxygen.json  
 extracting: Pubmed_Abstract/11.json  
 extracting: Pubmed_Abstract/13.json  
  inflating: Pubmed_Abstract/13_valent_pneumococcal_conjugate_vaccine.json  
 extracting: Pubmed_Abstract/15.json  
  inflating: Pubmed_Abstract/17a_hydroxylase.json  
  inflating: Pubmed_Abstract/194_200.json  
  inflating: Pubmed_Abstract/194_204.json  
  inflating: Pubmed_Abstract/199_200.json  
  inflating: Pubmed_Abstract/1_100.json  
  inflating: Pubmed_Abstract/1_1_mg_dL.json  
  inflating: Pubmed_Abstract/1_200.json  
  inflating: Pubmed_Abstract/1_34_to_2_36.json  
  inflating: Pubmed